In [1]:
import base64
import csv
import json
import os
import requests
import tempfile
import urllib
import pprint
pp = pprint.PrettyPrinter()

import tensorflow as tf

import tfx
from tfx.components.evaluator.component import Evaluator
from tfx.components.example_gen.csv_example_gen.component import CsvExampleGen
from tfx.components.example_validator.component import ExampleValidator
from tfx.components.model_validator.component import ModelValidator
from tfx.components.pusher.component import Pusher
from tfx.components.schema_gen.component import SchemaGen
from tfx.components.statistics_gen.component import StatisticsGen
from tfx.components.trainer.component import Trainer
from tfx.components.transform.component import Transform
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import evaluator_pb2
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2

import tensorflow_model_analysis as tfma

In [2]:
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.8.0
TFX version: 1.7.1


In [3]:
DATA_ROOT_DIR='data'
MODULE_FILE='trainer.py'
NOTEBOOK_FILEPATH='Pipeline_Development.ipynb'
PIPELINE_EXPORT_FILEPATH='pipeline_output/pipeline.py'
MODEL_EXPORT_FILEPATH='model_output'
PIPELINE_ROOT = os.path.join('pipeline_artifacts')
METADATA_PATH = os.path.join('metadata', 'metadata.db')

In [4]:
context = InteractiveContext(
    pipeline_root=PIPELINE_ROOT,
    metadata_connection_config=tfx.orchestration.metadata.sqlite_metadata_connection_config(METADATA_PATH)
)

In [5]:
example_gen = CsvExampleGen(input_base=DATA_ROOT_DIR)
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 9
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [6]:
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen)
context.show(statistics_gen.outputs['statistics'])

In [7]:
schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'])
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 11
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [8]:
validator_gen = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)
context.run(validator_gen)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 12
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [9]:
trainer_gen = Trainer(
    module_file=MODULE_FILE,
    examples=example_gen.outputs['examples'],
    train_args=trainer_pb2.TrainArgs(num_steps=2),
    eval_args=trainer_pb2.EvalArgs(num_steps=2)
)
context.run(trainer_gen)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying features.py -> build/lib
copying model.py -> build/lib
copying trainer.py -> build/lib
installing to /var/folders/yt/mm_9lw0j7nbb6ynlflgdw66h0000gn/T/tmp8tr_xkh4
running install
running install_lib
copying build/lib/features.py -> /var/folders/yt/mm_9lw0j7nbb6ynlflgdw66h0000gn/T/tmp8tr_xkh4
copying build/lib/model.py -> /var/folders/yt/mm_9lw0j7nbb6ynlflgdw66h0000gn/T/tmp8tr_xkh4
copying build/lib/trainer.py -> /var/folders/yt/mm_9lw0j7nbb6ynlflgdw66h0000gn/T/tmp8tr_xkh4
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifes

/Users/joe/opt/anaconda3/envs/tfx/lib/python3.7/site-packages/setuptools/command/install.py:37: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  setuptools.SetuptoolsDeprecationWarning,


Processing ./pipeline_artifacts/_wheels/tfx_user_code_Trainer-0.0+87f21a12e91bb0469325cbfc50747698f8decff25ef453c73a51e7f46d5a6553-py3-none-any.whl
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


...

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
INFO:tensorflow:Assets written to: pipeline_artifacts/Trainer/model/13/Format-Serving/assets


INFO:tensorflow:Assets written to: pipeline_artifacts/Trainer/model/13/Format-Serving/assets


ExecutionResult(
    component_id: Trainer
    execution_id: 13
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

In [11]:
from features import FEATURE_KEYS, LABEL_KEY


eval_config = tfma.EvalConfig(
    model_specs=[
        # This assumes a serving model with signature 'serving_default'. If
        # using estimator based EvalSavedModel, add signature_name='eval' and
        # remove the label_key. Note, if using a TFLite model, then you must set
        # model_type='tf_lite'.
        tfma.ModelSpec(label_key='<label_key>')
    ],
    metrics_specs=[
        tfma.MetricsSpec(
            # The metrics added here are in addition to those saved with the
            # model (assuming either a keras model or EvalSavedModel is used).
            # Any metrics added into the saved model (for example using
            # model.compile(..., metrics=[...]), etc) will be computed
            # automatically.
            metrics=[
                tfma.MetricConfig(class_name='ExampleCount'),
                tfma.MetricConfig(
                    class_name='BinaryAccuracy',
                    threshold=tfma.MetricThreshold(
                        value_threshold=tfma.GenericValueThreshold(
                            lower_bound={'value': 0.5}),
                        change_threshold=tfma.GenericChangeThreshold(
                            direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                            absolute={'value': -1e-10})))
            ]
        )
    ],
    slicing_specs=[
        # An empty slice spec means the overall slice, i.e. the whole dataset.
        tfma.SlicingSpec(),
        # Data can be sliced along a feature column. In this case, data is
        # sliced along feature column trip_start_hour.
        tfma.SlicingSpec(feature_keys=['trip_start_hour'])
    ])


evaluator_gen = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer_gen.outputs['model'],
    eval_config=eval_config
)
context.run(evaluator_gen)

ValueError: no value provided for label

This may be caused by a configuration error (i.e. label, and/or prediction keys were not specified) or an error in the pipeline.

to_label_prediction_example_weight(inputs={'labels': None, 'predictions': array([0.02993465], dtype=float32)}, eval_config=model_specs {
  label_key: "<label_key>"
}
slicing_specs {
}
slicing_specs {
  feature_keys: "trip_start_hour"
}
metrics_specs {
  metrics {
    class_name: "ExampleCount"
  }
  metrics {
    class_name: "BinaryAccuracy"
    threshold {
      value_threshold {
        lower_bound {
          value: 0.5
        }
      }
    }
  }
  model_names: ""
}
, model_name=, output_name=, sub_key=None, aggregation_type=None, class_weights=None, fractional_labels=False, flatten=False, squeeze=True, allow_none=False) [while running 'ExtractEvaluateAndWriteResults/ExtractAndEvaluate/EvaluateMetricsAndPlots/ComputeMetricsAndPlots()/CombineMetricsPerSlice/WindowIntoDiscarding']

In [ ]:
pusher_gen = Pusher(
    model_blessing=evaluator_gen.outputs['blessing'],
    model=trainer_gen.outputs['model'],
    push_destination=tfx.proto.pusher_pb2.PushDestination(
        filesystem=tfx.proto.pusher_pb2.PushDestination.Filesystem(
            base_directory=MODEL_EXPORT_FILEPATH
        )
    )
)
context.run(pusher_gen)

In [ ]:
context.export_to_pipeline(notebook_filepath=NOTEBOOK_FILEPATH,
                           export_filepath=PIPELINE_EXPORT_FILEPATH,
                           runner_type='beam')